In [89]:
import numpy as np
import pandas as pd
import csv
import operator
import sys
import re
import requests
import bs4
import os
import pylab

coachName = 'Don Shula'

##below functions courtesy of BenKite on GitHub: https://github.com/BenKite/football_data/blob/master/profootballReferenceScrape.py
def findTables(url):
    res = requests.get(url)
    comm = re.compile("<!--|-->")
    soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)
            
def pullCoachRankTable(url, tableID, header = True):
    cols = ['Year','TmCode','Role','Tms','Overall_WL%_Rank','Overall_T/G_Rank','Overall_PtsDiff_Rank',
            'Overall_YdsDiff_Rank','Offense_Yds_Rank','Offense_Pts_Rank','Offense_GvA_Rank','Rushing Off_Att_Rank',
            'Rushing Off_Yds_Rank','Rushing Off_TD_Rank','Rushing Off_Y/A_Rank','Rushing Off_FL_Rank',
            'Passing Off_Att_Rank','Passing Off_Yds_Rank','Passing Off_TD_Rank','Passing Off_Int_Rank',
            'Passing Off_NY/A_Rank','Defense_Yds_Rank','Defense_Pts_Rank','Defense_TkA_Rank',
            'Rushing Def_Att_Rank','Rushing Def_Yds_Rank','Rushing Def_TD_Rank','Rushing Def_Y/A_Rank',
            'Rushing Def_FR_Rank','Passing Def_Att_Rank','Passing Def_Yds_Rank','Passing Def_TD_Rank',
            'Passing Def_Int_Rank','Passing Def_netY/A_Rank',
]
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    data_rows = tables[0].findAll('tr')
    game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
        for i in range(len(data_rows))
        ]
    data = pd.DataFrame(game_data, columns = cols)
    data = data.drop([data.index[0] , data.index[1]])
    if header == True:
        data_header = tables[0].findAll('thead')
        data_header = data_header[0].findAll("tr")
        data_header = data_header[0].findAll("th")
        header = []
        for i in range(len(data.columns)):
            header.append(data_header[i].getText())
        data.columns = header
        data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    data = data.replace(r'^\s*$', np.nan, regex=True)
    data = data[data['Year'].str.contains('yr')==False] 
    data['Name'] = coachName
    return(data)          

def pullCoachResultsTable(url, tableID, header = True):
    cols = ['Year', 'Age', 'Tm', 'Lg', 'G', 'W', 'L', 'T', 'W-L%_1', 'SRS', 'OSRS',
    'DSRS', 'G plyf', 'W plyf', 'L plyf', 'W-L%_2','Num','Won','Rank','Notes']
    res = requests.get(url)
    ## Work around comments
    comm = re.compile("<!--|-->")
    soup = bs4.BeautifulSoup(comm.sub("", res.text), 'lxml')
    tables = soup.findAll('table', id = tableID)
    data_rows = tables[0].findAll('tr')
    game_data = [[td.getText() for td in data_rows[i].findAll(['th','td'])]
        for i in range(len(data_rows))]
    data = pd.DataFrame(game_data)
    data = data.drop([data.index[0] , data.index[1]])
    if len(data.columns) == 18:
        data.insert(loc = 16, column='Num', value = np.nan)
        data.insert(loc = 17, column='Won', value = np.nan)
        data.columns = cols
    elif len(data.columns) == 20:
        data.columns = cols
    else:
        raise Exception(print(len(data.columns)))
    if header == True:
        data_header = tables[0].findAll('thead')
        data_header = data_header[0].findAll("tr")
        data_header = data_header[0].findAll("th")
        header = []
        for i in range(len(data.columns)):
            header.append(data_header[i].getText())
        data.columns = header
        data = data.loc[data[header[0]] != header[0]]
    data = data.reset_index(drop = True)
    data = data.replace(r'^\s*$', np.nan, regex=True)
    data = data[data['Year'].str.contains('yr')==False]
    data['Name'] = coachName
    data['Role'] = 'HC'
    return(data)     

In [82]:
x = 'Don Shula'

findTables('https://www.pro-football-reference.com/coaches/BeliBi0.htm')

coachRank = pullCoachRankTable('https://www.pro-football-reference.com/coaches/' +'BeliBi0' +'.htm', 'coaching_ranks', header = False)
coachResults = pullCoachResultsTable('https://www.pro-football-reference.com/coaches/' + 'BeliBi0' +'.htm', 'coaching_results', header = False)
result = pd.merge(coachRank, coachResults, on=['Name','Year','Role'])

#coachRank.head()
coachResults.head(10000)
#result.head(1000)


,Year,Age,Tm,Lg,G,W,L,T,W-L%_1,SRS,...,G plyf,W plyf,L plyf,W-L%_2,Num,Won,Rank,Notes,Name,Role
0,1991,39,Cleveland Browns,NFL,16,6,10,0,.375,-0.4,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,Don Shula,HC
1,1992,40,Cleveland Browns,NFL,16,7,9,0,.438,-1.0,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,Don Shula,HC
2,1993,41,Cleveland Browns,NFL,16,7,9,0,.438,-1.2,...,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,Don Shula,HC
3,1994,42,Cleveland Browns,NFL,16,11,5,0,.688,7.3,...,2,1,1,.500,2,NaN,NaN,NaN,Don Shula,HC
4,1995,43,Cleveland Browns,NFL,16,5,11,0,.313,-3.8,...,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,Don Shula,HC
5,2000,48,New England Patriots,NFL,16,5,11,0,.313,-2.5,...,NaN,NaN,NaN,NaN,5,4,0,NaN,Don Shula,HC
6,2001,49,New England Patriots,NFL,16,11,5,0,.688,4.3,...,3,3,0,1.000,1,6,3,Super Bowl Champions,Don Shula,HC
7,2002,50,New England Patriots,NFL,16,9,7,0,.563,4.0,...,NaN,NaN,NaN,NaN,2,9,5,NaN,Don Shula,HC
8,2003,51,New England Patriots,NFL,16,14,2,0,.875,6.9,...,3,3,0,1.000,1,8,1,Super Bowl Champions,Don Shula,HC
9,2004,52,New England Patriots,NFL,16,14,2,0,.875,12.8,...,3,3,0,1.000,1,8,5,Super Bowl Champions,Don Shula,HC


In [90]:
df = pd.read_csv('coaches2019.csv')  # doctest: +SKIP
coachArray = df.Coach.unique()
dfCoach = pd.DataFrame(columns = ['Year', 'TmCode', 'Role', 'Tms', 'Overall_WL%_Rank', 'Overall_T/G_Rank',
       'Overall_PtsDiff_Rank', 'Overall_YdsDiff_Rank', 'Offense_Yds_Rank',
       'Offense_Pts_Rank', 'Offense_GvA_Rank', 'Rushing Off_Att_Rank',
       'Rushing Off_Yds_Rank', 'Rushing Off_TD_Rank', 'Rushing Off_Y/A_Rank',
       'Rushing Off_FL_Rank', 'Passing Off_Att_Rank', 'Passing Off_Yds_Rank',
       'Passing Off_TD_Rank', 'Passing Off_Int_Rank', 'Passing Off_NY/A_Rank',
       'Defense_Yds_Rank', 'Defense_Pts_Rank', 'Defense_TkA_Rank',
       'Rushing Def_Att_Rank', 'Rushing Def_Yds_Rank', 'Rushing Def_TD_Rank',
       'Rushing Def_Y/A_Rank', 'Rushing Def_FR_Rank', 'Passing Def_Att_Rank',
       'Passing Def_Yds_Rank', 'Passing Def_TD_Rank', 'Passing Def_Int_Rank',
       'Passing Def_netY/A_Rank', 'Name', 'Age', 'Tm', 'Lg', 'G', 'W', 'L',
       'T', 'W-L%_1', 'SRS', 'OSRS', 'DSRS', 'G plyf', 'W plyf', 'L plyf',
       'W-L%_2', 'Num', 'Won', 'Rank', 'Notes'])

errorArr = [['Norm Van Brocklin+','VanBNo0'],[ 'Jim Lee Howell','HoweJi0'],[ "Bill O'Brien",'OBriBi0'],[ 'Pop Ivy','IvyxPo0'],[ 'Red Hickey','HickHo0'],[ 'Red Strader','StraNo0'],[ 'Bo McMillin','McMiAl0'],[ 'Lone Star Dietz','DietLo0'],[ 'Buster Ramsey','RamsGa0'],[ 'Frank Gansz','GanzFr0'],[ 'Dick MacPherson','McPhDi0'],[ 'Bulldog Turner+','TurnCl0'],[ 'Ray Willsey','WilsRa0'],[ 'Bill Conkright','ConkRe0'],[ "Fred O'Connor",'OConFr0'],[ 'Jack Del Rio', 'DelRJa0'],[ 'Buddy Parker', 'ParkRa0'],[ 'Buck Shaw', 'ShawLa0']]
dfError = pd.DataFrame(errorArr, columns=['Name', 'CoachCode']) 
coachCodeArr = []

In [91]:
###Getting the array of codes used in the URL address of each coach's individual page


for x in coachArray:
    coachName = x 
    nameSplit = coachName.split()
    firstName = nameSplit[0][:2]
    lastName = nameSplit[1][:4]
    z = dfError["Name"].str.contains(coachName).sum()
    if z>0:
        unique_index = pd.Index(dfError['Name'])
        y = unique_index.get_loc(coachName)
        coachCode = errorArr[y][1]
        coachCodeArr.append(coachCode)       
    elif len(lastName) < 4:
        lastName = lastName + nameSplit[1][2]
        coachCode = lastName + firstName + '0'
        coachCodeArr.append(coachCode)
    else:
        coachCode = lastName + firstName + '0'
        coachCodeArr.append(coachCode)     
        
        
if len(coachCodeArr) != len(coachArray):        
    print('ERROR: The following is TRUE len(coachCodeArr) != len(coachArray) ')
    sys.exit()

In [92]:


a = 0
for x in coachCodeArr:
    coachName = coachArray[a]
    url = 'https://www.pro-football-reference.com/coaches/' + x + '.htm'
    coachRank = pullCoachRankTable(url, 'coaching_ranks', header = False)
    coachResults = pullCoachResultsTable(url, 'coaching_results', header = False)
    result = pd.merge(coachRank, coachResults, on=['Name','Year','Role'])   
    dfCoach = dfCoach.append(result, sort =False) 
    a = a+1

dfCoach = dfCoach.reset_index()
dfCoach.head(1000)

,index,Year,TmCode,Role,Tms,Overall_WL%_Rank,Overall_T/G_Rank,Overall_PtsDiff_Rank,Overall_YdsDiff_Rank,Offense_Yds_Rank,...,OSRS,DSRS,G plyf,W plyf,L plyf,W-L%_2,Num,Won,Rank,Notes
0,0,1963,BAL,HC,14,7,9,7,5,2,...,0.8,1.2,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
1,1,1964,BAL,HC,14,1,1,1,2,1,...,9.3,6.2,1,0,1,.000,NaN,NaN,1,NaN
2,2,1965,BAL,HC,14,2,8,2,5,5,...,7.0,5.9,1,0,1,.000,NaN,NaN,1,NaN
3,3,1966,BAL,HC,15,3,7,4,7,5,...,2.2,5.5,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
4,4,1967,BAL,HC,16,1,2,2,1,1,...,6.4,6.8,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
5,5,1968,BAL,HC,16,1,5,1,2,4,...,8.0,9.9,3,2,1,.667,NaN,NaN,1,NFL Champions
6,6,1969,BAL,HC,16,6,16,7,10,4,...,0.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
7,7,1970,MIA,HC,26,4,7,8,14,11,...,-1.3,0.0,1,0,1,.000,NaN,NaN,2,NaN
8,8,1971,MIA,HC,26,3,7,3,4,5,...,2.4,5.3,3,2,1,.667,NaN,NaN,1,AFC Champions
9,9,1972,MIA,HC,26,1,2,1,1,1,...,4.5,6.5,3,3,0,1.000,NaN,NaN,1,Super Bowl Champions


In [87]:
dfCoach.to_csv(r'dfCoach.csv')